In [1]:
import pandas as pd
import numpy as np

In [2]:
%run ../Jane/extract_data.py

In [4]:
nyc_zillow = zillow_data[zillow_data.City == 'New York'].copy()
nyc_zipcodes = zillow_data[zillow_data.City == 'New York'].Zipcode.value_counts().reset_index()['index'].to_list()
nyc_airqual = airqual_data[airqual_data['City'] == 'New York'].copy()
nyc_persinc = persinc_data[persinc_data.City == 'New York'].copy()
nyc_census = census_data[census_data.Zipcode.isin(nyc_zipcodes)].copy()

In [19]:
nyc_zillow.drop(['RegionID','Metro'],axis=1,inplace=True)

In [49]:
# reshape for better merging
inclvl_moderate = inclvl_data[inclvl_data.income_level_group == 'Moderate'][['Date','Vol']]
inclvl_moderate = inclvl_moderate.rename(columns={'Vol':'Vol_moderate_income'})
inclvl_low = inclvl_data[inclvl_data.income_level_group == 'Low'][['Date','Vol']]
inclvl_low = inclvl_low.rename(columns={'Vol':'Vol_low_income'})
inclvl_data = pd.merge(inclvl_moderate,inclvl_low,on='Date')
inclvl_data.head()

,Date,Vol_moderate_income,Vol_low_income
0,2005-01-01,4.578106e+09,7.635135e+08
1,2005-02-01,4.563908e+09,7.558710e+08
2,2005-03-01,4.762458e+09,8.171551e+08
3,2005-04-01,4.744005e+09,7.854169e+08
4,2005-05-01,4.752915e+09,8.289172e+08


In [51]:
df = pd.merge(nyc_zillow, inclvl_data,how='left',on='Date')

In [53]:
nyc_airqual.drop(['State','City'],axis=1,inplace=True)

In [57]:
nyc_airqual # only has data for 2 of the 5 counties :( -> omit :/

,County,AQIMean,Date
26386,Bronx,25.555341,2010-01-01
26387,Bronx,24.637082,2010-02-01
26388,Bronx,25.338308,2010-03-01
26389,Bronx,22.618458,2010-04-01
26390,Bronx,21.597693,2010-05-01
...,...,...,...
27068,Queens,8.975915,2020-05-01
27069,Queens,9.439863,2020-06-01
27070,Queens,8.190798,2020-07-01
27071,Queens,11.708356,2020-08-01


In [58]:
df

,Zipcode,City,State,County,SizeRank,Date,Rent,Year,Vol_moderate_income,Vol_low_income
0,10025,New York,NY,New York County,1,2010-09-01,2930.0,2010,2.150781e+09,3.727692e+08
1,10023,New York,NY,New York County,3,2010-09-01,2797.0,2010,2.150781e+09,3.727692e+08
2,10002,New York,NY,New York County,7,2010-09-01,3083.0,2010,2.150781e+09,3.727692e+08
3,11226,New York,NY,Kings County,11,2010-09-01,1717.0,2010,2.150781e+09,3.727692e+08
4,10467,New York,NY,Bronx County,12,2010-09-01,1347.0,2010,2.150781e+09,3.727692e+08
...,...,...,...,...,...,...,...,...,...,...
16945,11436,New York,NY,Queens County,1820,2020-01-01,2057.0,2020,NaN,NaN
16946,11366,New York,NY,Queens County,1826,2020-01-01,2185.0,2020,NaN,NaN
16947,11109,New York,NY,New York County,1828,2020-01-01,3325.0,2020,NaN,NaN
16948,10307,New York,NY,Richmond County,1829,2020-01-01,1536.0,2020,NaN,NaN


In [96]:
nyc_persinc

,City,Year,PersonalIncome
2,New York,2008,48585
6,New York,2009,46606
10,New York,2010,46919
14,New York,2011,48043
18,New York,2012,49407
22,New York,2013,49511
26,New York,2014,51102
30,New York,2015,52904
34,New York,2016,53694
38,New York,2017,55716


In [88]:
acs = nyc_census[['do_date','total_pop','households','median_age','median_income','income_per_capita','pop_determined_poverty_status',
                  'poverty','gini_index','housing_units','different_house_year_ago_different_city','different_house_year_ago_same_city',
                  'aggregate_travel_time_to_work','bachelors_degree','employed_pop','unemployed_pop',
                  'employed_arts_entertainment_recreation_accommodation_food','Zipcode']].copy()

In [89]:
# proportion of 
acs['pct_poverty'] = acs['poverty']/acs['pop_determined_poverty_status']
acs.drop(['poverty','pop_determined_poverty_status'],axis=1,inplace=True)

In [90]:
acs['housing_availability'] = acs['housing_units']/acs['households']
acs['home_density'] = acs['total_pop']/acs['households']

acs['pct_employed'] = acs['employed_pop']/acs['total_pop']
acs['pct_jobs_nightlife'] = acs['employed_arts_entertainment_recreation_accommodation_food']/acs['employed_pop']
acs['pct_unemployed'] = acs['unemployed_pop']/acs['total_pop']

acs['move_within_city'] = acs['different_house_year_ago_same_city']/acs['total_pop']
acs['move_new_city'] = acs['different_house_year_ago_different_city']/acs['total_pop']
acs['avg_commute_time'] = acs['aggregate_travel_time_to_work'] / acs['employed_pop']

acs.drop(['housing_units','employed_pop','employed_arts_entertainment_recreation_accommodation_food','unemployed_pop'],axis=1,inplace=True)
acs.drop(['different_house_year_ago_same_city','different_house_year_ago_different_city','aggregate_travel_time_to_work'],axis=1,inplace=True)

In [92]:
acs['pct_college'] = acs['bachelors_degree'] / acs['total_pop']
acs.drop('bachelors_degree',axis=1,inplace=True)

In [94]:
acs

,do_date,total_pop,households,median_age,median_income,income_per_capita,gini_index,Zipcode,pct_poverty,housing_availability,home_density,pct_employed,pct_jobs_nightlife,pct_unemployed,move_within_city,move_new_city,avg_commute_time,pct_college
2440,2014-01-01,21896,8598,44.9,85707.0,49313.0,0.5057,10471,0.100143,1.124448,2.546639,0.457618,0.082735,0.035440,0.051836,0.047543,41.786427,0.168341
2699,2014-01-01,67040,22267,35.6,65900.0,30110.0,0.4653,11210,0.124258,1.083577,3.010733,0.460069,0.056415,0.031578,0.047181,0.005892,43.187919,0.132354
2752,2014-01-01,27602,8812,33.8,56661.0,26865.0,0.4612,11232,0.216478,1.053904,3.132320,0.487175,0.180115,0.034092,0.078835,0.036374,39.371607,0.129338
2890,2014-01-01,18929,5558,40.8,73699.0,26603.0,0.4006,11428,0.084435,1.074667,3.405721,0.487189,0.076122,0.029690,0.055259,0.006445,44.567339,0.091711
2979,2014-01-01,24037,7500,42.0,74581.0,32689.0,0.4605,11427,0.084994,1.064400,3.204933,0.466780,0.082353,0.025627,0.055872,0.005866,43.923351,0.141241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27091,2014-01-01,95666,31835,40.3,81399.0,34686.0,0.4381,11234,0.080064,1.066342,3.005057,0.465129,0.068836,0.035938,0.041446,0.005289,45.273052,0.142485
27103,2014-01-01,56308,24079,34.2,61121.0,38741.0,0.5133,11216,0.173683,1.132647,2.338469,0.565586,0.106572,0.038787,0.107676,0.025769,38.641787,0.218992
27107,2014-01-01,83615,30946,40.8,60873.0,31040.0,0.4788,11229,0.146091,1.069734,2.701965,0.452766,0.079719,0.029720,0.059475,0.007786,44.370543,0.159971
27109,2014-01-01,91972,33240,34.1,37243.0,20285.0,0.5160,11207,0.288941,1.121089,2.766907,0.417181,0.075790,0.044340,0.056235,0.010938,44.067216,0.072065
